In [1]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
image_width, image_height = 64, 64
num_classes = 164
images = []
labels = []
class_label_mapping = {}
class_count = 0

In [3]:
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((image_width, image_height))
    img = np.array(img)
    img = img / 255.0 
    return img

In [4]:
dataset_path = r'D:\Project3\tai\EarVN1.0 dataset\Images'

In [5]:
for class_name in os.listdir(dataset_path):
    class_folder = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_folder):
        continue
    class_label_mapping[class_name] = class_count
    class_count += 1

    for image_name in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_name)
        img = preprocess_image(image_path)
        images.append(img)
        labels.append(class_label_mapping[class_name])


In [6]:
images = np.array(images)
labels = np.array(labels)
labels = to_categorical(labels, num_classes)

In [7]:
indices = np.arange(images.shape[0])
np.random.shuffle(indices)
images = images[indices]
labels = labels[indices]

In [8]:
train_images, temp_images, train_labels, temp_labels = train_test_split(images, labels, test_size=0.4, random_state=42)
val_images, test_images, val_labels, test_labels = train_test_split(temp_images, temp_labels, test_size=0.5, random_state=42)

In [9]:
train_datagen1 = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [10]:
train_datagen2 = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator()

In [11]:
batch_size = 32
epochs = 100

In [13]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))

94765736/94765736 [==============================] - 5s 0us/step


In [14]:
for layer in resnet_model.layers:
    layer.trainable = False

In [15]:
model = Sequential()
model.add(resnet_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2, 2, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 512)               4194816   
                                                                 
 dense_1 (Dense)             (None, 164)               84132     
                                                                 
Total params: 27,866,660
Trainable params: 4,278,948
Non-trainable params: 23,587,712
_________________________________________________________________


In [17]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_generator1 = train_datagen1.flow(train_images, train_labels, batch_size=batch_size)
val_generator = val_datagen.flow(val_images, val_labels, batch_size=batch_size)

In [18]:
model.fit(train_generator1, epochs=epochs, validation_data= val_generator)

Epoch 1/100
533/533 [==============================] - 23s 30ms/step - loss: 5.1070 - accuracy: 0.0082 - val_loss: 5.0976 - val_accuracy: 0.0100
Epoch 2/100
533/533 [==============================] - 15s 29ms/step - loss: 5.0942 - accuracy: 0.0097 - val_loss: 5.0964 - val_accuracy: 0.0111
Epoch 3/100
533/533 [==============================] - 15s 28ms/step - loss: 5.0915 - accuracy: 0.0096 - val_loss: 5.0960 - val_accuracy: 0.0111
Epoch 4/100
533/533 [==============================] - 15s 28ms/step - loss: 5.0898 - accuracy: 0.0095 - val_loss: 5.0961 - val_accuracy: 0.0111
Epoch 5/100
533/533 [==============================] - 15s 29ms/step - loss: 5.0887 - accuracy: 0.0096 - val_loss: 5.0963 - val_accuracy: 0.0111
Epoch 6/100
533/533 [==============================] - 15s 29ms/step - loss: 5.0882 - accuracy: 0.0101 - val_loss: 5.0967 - val_accuracy: 0.0111
Epoch 7/100
533/533 [==============================] - 15s 29ms/step - loss: 5.0877 - accuracy: 0.0097 - val_loss: 5.0970 - val_ac

In [19]:
train_generator2 = train_datagen2.flow(train_images, train_labels, batch_size=batch_size)

In [20]:
model.fit(train_generator2, epochs=epochs, validation_data=val_generator)

Epoch 1/100
533/533 [==============================] - 16s 30ms/step - loss: 5.0044 - accuracy: 0.0149 - val_loss: 4.9815 - val_accuracy: 0.0183
Epoch 2/100
533/533 [==============================] - 15s 29ms/step - loss: 5.0061 - accuracy: 0.0150 - val_loss: 4.9671 - val_accuracy: 0.0180
Epoch 3/100
533/533 [==============================] - 16s 30ms/step - loss: 4.9998 - accuracy: 0.0157 - val_loss: 4.9642 - val_accuracy: 0.0164
Epoch 4/100
533/533 [==============================] - 15s 28ms/step - loss: 5.0028 - accuracy: 0.0141 - val_loss: 4.9630 - val_accuracy: 0.0165
Epoch 5/100
533/533 [==============================] - 15s 28ms/step - loss: 5.0059 - accuracy: 0.0140 - val_loss: 4.9803 - val_accuracy: 0.0165
Epoch 6/100
533/533 [==============================] - 16s 30ms/step - loss: 5.0049 - accuracy: 0.0130 - val_loss: 4.9702 - val_accuracy: 0.0176
Epoch 7/100
533/533 [==============================] - 15s 28ms/step - loss: 5.0024 - accuracy: 0.0150 - val_loss: 4.9807 - val_ac

In [21]:
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

178/178 [==============================] - 3s 17ms/step - loss: 4.8913 - accuracy: 0.0208
Test Loss: 4.891330242156982, Test Accuracy: 0.020763680338859558
